In [ ]:
import polars as pl
from fuzzywuzzy import process, fuzz

In [ ]:
def fuzzymergy(gateway, terms):
   tnc = terms.collect()['org_name'].to_list()
   test_str = ''
   print(process.extract(test_str, tnc, limit=1)[0][0])
   print(process.extract(test_str, tnc, limit=1)[0][1])

   result = (
      gateway
      .collect()
      .with_columns(
         pl.col('licensee_clean').map_elements(lambda x: process.extract(x, tnc, limit=1)[0][0]).alias('match'),
         pl.col('licensee_clean').map_elements(lambda x: process.extract(x, tnc, limit=1)[0][1]).alias('score')
      )
    )
   return result
    

In [ ]:
terms = (
    pl.scan_csv("data/terms.csv", infer_schema_length=10000)
    .select('org_name')
)


In [ ]:
gateway = (
    pl.scan_csv('data/gateway_connections.csv', infer_schema_length=10000)
    .with_columns(
        pl.col('Licensee').str.replace_all(r'\([^)]*\)', '').str.to_uppercase().str.strip_chars(' )').alias('licensee_clean'),
    )
    .select('licensee_clean')
)


In [ ]:
fuzzymergy(gateway=gateway, terms=terms).sort('score',descending=False).write_csv('data/checkywecky.csv')